In [1]:
## Black Sholes Martin model for valueing Call options 
## s0 = stock price at time zero
##K = strike price
## T = time to maturity
## r = risk free rate of return
## sigma = volatility
def bsm_call_value(s0, K, T, r, sigma):
    from math import log, sqrt, exp
    from scipy import stats
    d1 = (log(s0/K) + (r + 0.5*sigma**2)*T) / (sigma*sqrt(T))
    d2 = (log(s0/K) + (r - 0.5*sigma**2)*T) / (sigma*sqrt(T))
    value = (s0*stats.norm.cdf(d1,0.0,1.0) - K*exp(-r*T)*stats.norm.cdf(d2,0.0,1.0))
    return value

In [2]:
bsm_call_value(100,125,1,0.05,0.2)

2.324274466015236

In [3]:
## Function to calculate vega for the calll option
## s0 = stock price at time zero
##K = strike price
## T = time to maturity
## r = risk free rate of return
## sigma = volatility
def bsm_vega(s0, K, T, r, sigma):
    from math import log, sqrt
    from scipy import stats
    d1 = (log(s0/K) + (r + 0.5*sigma**2)*T) / (sigma*sqrt(T))
    vega = s0*stats.norm.cdf(d1,0.0,1.0)*sqrt(T)
    return vega

In [4]:
bsm_vega(100,125,1,0.05,0.2)

22.19221296481797

In [5]:
## Function to calculate Implied Volatility of the Call Options
## s0 = stock price at time zero
##K = strike price
## T = time to maturity
## r = risk free rate of return
## sigma = volatility
## it = no. of iterations
def bsm_call_impvol(s0, K, T, r, C0, sigma_est, it = 100):
    for i in range(it):
        sigma_est -= ((bsm_call_value(s0, K, T, r, sigma_est) - C0) / bsm_vega(s0, K, T, r, sigma_est))
        return sigma_est
    

In [6]:
bsm_call_impvol(100,125,1,0.05,2.32,2)

1.2294301519023478

In [7]:
bsm_call_impvol(100,125,1,0.05,2.32,2,200)

1.2294301519023478

In [8]:
bsm_call_impvol(100,125,1,0.05,2.32,0.2,100)

0.19980738892412342

In [10]:
v0 = 17.6639
r = 0.01
import pandas as pd
od = pd.read_csv('C:/Users/amitb/Compressed/AAPL_OptionTrades_NBBO/AAPL_OptionTrades_NBBO.csv')
od.head()

,Symbol,Exchange,Company_name,Trade_date,Trade_time,Option_trade_price,Trade_size,Trade_exchange,Trade_condition,Option_symbol,...,Ask_price,Ask_time,Ask_size,Ask_exchange,Underlying_bid_price,Underlying_bid_time,Underlying_ask_price,Underlying_ask_time,Underlying_last_price,Underlying_last_time
0,AAPL,NASDAQ,Apple Inc,10/15/2019,09:30:00.196,18.44,1,Q,Auto,AAPL 191101C00220000,...,0.00,00:00:00.000,0,*,236.35,09:30:00.180,236.45,09:30:00.180,236.45,09:30:00.170
1,AAPL,NASDAQ,Apple Inc,10/15/2019,09:30:00.495,0.45,33,Y2,Auto,AAPL 191025P00220000,...,0.52,09:30:00.406,2,Y2,236.35,09:30:00.280,236.45,09:30:00.280,236.45,09:30:00.170
2,AAPL,NASDAQ,Apple Inc,10/15/2019,09:30:00.758,2.80,22,ME,Auto,AAPL 200117C00260000,...,2.80,09:30:00.758,22,ME,236.35,09:30:00.690,236.45,09:30:00.690,236.45,09:30:00.170
3,AAPL,NASDAQ,Apple Inc,10/15/2019,09:30:00.758,15.80,1,MI,Regular,AAPL 191220C00225000,...,18.50,09:30:00.681,2,ME,236.35,09:30:00.690,236.45,09:30:00.690,236.45,09:30:00.170
4,AAPL,NASDAQ,Apple Inc,10/15/2019,09:30:00.758,12.10,1,MI,Regular,AAPL 191220C00235000,...,12.10,09:30:00.675,10,MI,236.35,09:30:00.690,236.45,09:30:00.690,236.45,09:30:00.170


In [11]:
od.columns

Index(['Symbol', 'Exchange', 'Company_name', 'Trade_date', 'Trade_time',
       'Option_trade_price', 'Trade_size', 'Trade_exchange', 'Trade_condition',
       'Option_symbol', 'Option_expiration', 'Price_strike', 'Call_Put',
       'Style', 'Bid_price', 'Bid_time', 'Bid_size', 'Bid_exchange',
       'Ask_price', 'Ask_time', 'Ask_size', 'Ask_exchange',
       'Underlying_bid_price', 'Underlying_bid_time', 'Underlying_ask_price',
       'Underlying_ask_time', 'Underlying_last_price', 'Underlying_last_time'],
      dtype='object')

In [12]:
od = od[['Trade_date','Option_trade_price','Option_expiration','Price_strike','Call_Put','Underlying_last_price']]
od.head()

,Trade_date,Option_trade_price,Option_expiration,Price_strike,Call_Put,Underlying_last_price
0,10/15/2019,18.44,11/01/2019,220.0,C,236.45
1,10/15/2019,0.45,10/25/2019,220.0,P,236.45
2,10/15/2019,2.80,01/17/2020,260.0,C,236.45
3,10/15/2019,15.80,12/20/2019,225.0,C,236.45
4,10/15/2019,12.10,12/20/2019,235.0,C,236.45


In [13]:
callData = od.groupby('Call_Put').get_group('C')
callData.head()

,Trade_date,Option_trade_price,Option_expiration,Price_strike,Call_Put,Underlying_last_price
0,10/15/2019,18.44,11/01/2019,220.0,C,236.45
2,10/15/2019,2.80,01/17/2020,260.0,C,236.45
3,10/15/2019,15.80,12/20/2019,225.0,C,236.45
4,10/15/2019,12.10,12/20/2019,235.0,C,236.45
5,10/15/2019,7.05,10/18/2019,230.0,C,236.45


In [14]:
callData.dtypes

Trade_date                object
Option_trade_price       float64
Option_expiration         object
Price_strike             float64
Call_Put                  object
Underlying_last_price    float64
dtype: object

In [15]:
callData = callData.drop(columns = 'Call_Put', axis =1)
callData.head()

,Trade_date,Option_trade_price,Option_expiration,Price_strike,Underlying_last_price
0,10/15/2019,18.44,11/01/2019,220.0,236.45
2,10/15/2019,2.80,01/17/2020,260.0,236.45
3,10/15/2019,15.80,12/20/2019,225.0,236.45
4,10/15/2019,12.10,12/20/2019,235.0,236.45
5,10/15/2019,7.05,10/18/2019,230.0,236.45


In [16]:
callData.dtypes

Trade_date                object
Option_trade_price       float64
Option_expiration         object
Price_strike             float64
Underlying_last_price    float64
dtype: object

In [17]:
callData['Option_expiration'] = pd.to_datetime(callData['Option_expiration'])
callData['Trade_date'] = pd.to_datetime(callData['Trade_date'])

In [18]:
callData.dtypes

Trade_date               datetime64[ns]
Option_trade_price              float64
Option_expiration        datetime64[ns]
Price_strike                    float64
Underlying_last_price           float64
dtype: object

In [19]:
callData['TTM'] = callData['Option_expiration'] - callData['Trade_date']

In [20]:
callData.head()

,Trade_date,Option_trade_price,Option_expiration,Price_strike,Underlying_last_price,TTM
0,2019-10-15,18.44,2019-11-01,220.0,236.45,17 days
2,2019-10-15,2.80,2020-01-17,260.0,236.45,94 days
3,2019-10-15,15.80,2019-12-20,225.0,236.45,66 days
4,2019-10-15,12.10,2019-12-20,235.0,236.45,66 days
5,2019-10-15,7.05,2019-10-18,230.0,236.45,3 days


In [21]:
callData.dtypes

Trade_date                datetime64[ns]
Option_trade_price               float64
Option_expiration         datetime64[ns]
Price_strike                     float64
Underlying_last_price            float64
TTM                      timedelta64[ns]
dtype: object

In [22]:
callData['TTM'] = pd.to_numeric(callData['TTM'].dt.days, downcast='float')

In [23]:
for i in callData:
    callData['TTM'] = callData['TTM'] / 365
    

In [24]:
callData.head()

,Trade_date,Option_trade_price,Option_expiration,Price_strike,Underlying_last_price,TTM
0,2019-10-15,18.44,2019-11-01,220.0,236.45,7.189384e-15
2,2019-10-15,2.80,2020-01-17,260.0,236.45,3.975307e-14
3,2019-10-15,15.80,2019-12-20,225.0,236.45,2.791173e-14
4,2019-10-15,12.10,2019-12-20,235.0,236.45,2.791173e-14
5,2019-10-15,7.05,2019-10-18,230.0,236.45,1.268715e-15


In [25]:
callData.dtypes

Trade_date               datetime64[ns]
Option_trade_price              float64
Option_expiration        datetime64[ns]
Price_strike                    float64
Underlying_last_price           float64
TTM                             float32
dtype: object

In [26]:
##bsm_call_impvol(s0, K, T, r, C0, sigma_est, it = 100)

r = 0.01


sigma_est = 0.01
#for i in callData:
#    callData['Imp_vol'] = bsm_call_impvol(s0, K, x, r, C0, sigma_est)

callData['r'] = [0.01 for i in range(len(callData))]
callData['sigma_est'] = [0.01 for i in range(len(callData))]

callData['Imp_vol'] = list(map(bsm_call_impvol, callData['Underlying_last_price'],
                              callData['Price_strike'],
                              callData['TTM'],
                              callData['r'],
                              callData['Option_trade_price'],
                              callData['sigma_est']))

C:\Users\amitb\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in double_scalars
  after removing the cwd from sys.path.


In [27]:
callData['Imp_vol'].head()

0    9.925856e+04
2             inf
3    1.101176e+05
4    2.695982e+05
5    7.124099e+04
Name: Imp_vol, dtype: float64

In [28]:
callData['Imp_vol']

0        9.925856e+04
2                 inf
3        1.101176e+05
4        2.695982e+05
5        7.124099e+04
6        2.114645e+05
8                 inf
12                inf
20       1.061034e+04
21       8.700484e+03
24                inf
25                inf
26                inf
27                inf
30      -7.362741e+03
31                inf
34       2.006961e+05
35                inf
36                inf
40       1.086124e+05
41       2.390082e+05
44                inf
45                inf
49       9.826929e+04
50                inf
51                inf
52       3.682031e+04
53       1.813123e+05
55       9.264463e+04
56       2.137954e+04
             ...     
60578    2.123271e+05
60580             inf
60581             inf
60583    2.123361e+05
60584    7.337453e+04
60585    2.123361e+05
60586             inf
60587    2.135380e+05
60588             inf
60591    2.135380e+05
60592    2.135380e+05
60594    2.135380e+05
60595    2.111342e+05
60596    2.111342e+05
60597     

In [29]:
callData.head()

,Trade_date,Option_trade_price,Option_expiration,Price_strike,Underlying_last_price,TTM,r,sigma_est,Imp_vol
0,2019-10-15,18.44,2019-11-01,220.0,236.45,7.189384e-15,0.01,0.01,9.925856e+04
2,2019-10-15,2.80,2020-01-17,260.0,236.45,3.975307e-14,0.01,0.01,inf
3,2019-10-15,15.80,2019-12-20,225.0,236.45,2.791173e-14,0.01,0.01,1.101176e+05
4,2019-10-15,12.10,2019-12-20,235.0,236.45,2.791173e-14,0.01,0.01,2.695982e+05
5,2019-10-15,7.05,2019-10-18,230.0,236.45,1.268715e-15,0.01,0.01,7.124099e+04
